In [2]:
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
from data_fetch import fetch
from technical_indicator import atr, rvol, price_deviation


# Display floats with 2 decimal places
pd.options.display.float_format = "{:.2f}".format

SYMBOL = "AAPL"
TZ = "America/New_York"

HIST_DATA_INTERVAL = "1d"
HIST_DATA_PERIOD   = "1y"
HIST_DATA_PREPOST  = False

ATR_LOOKBACK_DAYS      = 14
RVOL_LOOKBACK_DAYS     = 20
PRICEDEV_LOOKBACK_DAYS = 20
RVOL_ALPHA = 0.5   # weight for SMA in hybrid RVOL (0 = pure EWM, 1 = pure SMA)

MARKET_DATA_INTERVAL = "5m"
MARKET_DATA_PERIOD   = "60d"
MARKET_DATA_PREPOST  = False

# ==========================
# Build Indicators
# ==========================
def build_indicators(raw_df) -> pd.DataFrame:
    """
    Fetch raw data and compute indicators: ATR, RVOL, Price Deviation.

    Returns
    -------
    DataFrame with columns [Close, Volume, ATR, RVOL, PriceDev]
    """
    atr_series   = atr(raw_df, ATR_LOOKBACK_DAYS)
    rvol_series  = rvol(raw_df, RVOL_LOOKBACK_DAYS,
                        method="hybrid", alpha=RVOL_ALPHA)
    sigma_series = price_deviation(raw_df, PRICEDEV_LOOKBACK_DAYS)

    combined_df = pd.concat(
        [raw_df["Close"], raw_df["Volume"], atr_series,
         rvol_series, sigma_series],
        axis=1
    )
    combined_df.columns = ["Close", "Volume", "ATR", "Hist_RVOL", "Price_Sigma"]

    return combined_df

In [3]:
## Get Historical Data
hist_df = pd.read_csv("data/CRCL_daily.csv")
hist_df["date"] = pd.to_datetime(hist_df["date"])
hist_df.set_index("date", inplace=True)
hist_df.columns = ["Open", "High", "Low", "Close", "Volume"]
hist_TI_df = build_indicators(hist_df)

In [4]:
## Get minute level data
min_df = pd.read_csv("data/CRCL_min.csv")
min_df["date"] = pd.to_datetime(min_df["date"])
min_df.set_index("date", inplace=True)
min_df.columns = ["Open", "High", "Low", "Close", "Volume"]
min_df = min_df[["Close", "Volume"]]


In [5]:
min_df["Time"] = min_df.index.time
min_df["Date"] = min_df.index.date

min_df["CumVolume"] = min_df.groupby("Date")["Volume"].cumsum()
daily_tot = min_df.groupby("Date")["Volume"].transform("sum").replace(0, pd.NA)
min_df["CumVolumeShare"] = min_df["CumVolume"] / daily_tot

pivot_cum_sum = min_df.pivot_table(index="Date", columns="Time", values="CumVolume")
rolling_avg_vol = pivot_cum_sum.rolling(window=10, min_periods=1).mean()

pivot_cum_share = min_df.pivot_table(index="Date", columns="Time", values="CumVolumeShare")
rolling_avg_share = pivot_cum_share.rolling(window=10, min_periods=1).mean()

avg_cumvol = rolling_avg_vol.stack().reset_index()
# 1. Combine into one column
avg_cumvol["Datetime"] = pd.to_datetime(
    avg_cumvol["Date"].astype(str) + " " + avg_cumvol["Time"].astype(str)
)

# 2. Set as index
avg_cumvol = avg_cumvol.set_index("Datetime")

# (optional) drop the old Date/Time columns
avg_cumvol = avg_cumvol.drop(columns=["Date", "Time"])
avg_cumvol.columns = ["Avg_CumVolume"]
avg_cumvol

,Avg_CumVolume
Datetime,
2025-07-21 04:00:00,7658.00
2025-07-21 04:01:00,13850.00
2025-07-21 04:02:00,19291.00
2025-07-21 04:03:00,23872.00
2025-07-21 04:04:00,32278.00
...,...
2025-08-18 19:55:00,16424779.70
2025-08-18 19:56:00,16426650.70
2025-08-18 19:57:00,16428987.50


In [6]:
min_df

,Close,Volume,Time,Date,CumVolume,CumVolumeShare
date,,,,,,
2025-07-21 04:00:00,224.49,7658.00,04:00:00,2025-07-21,7658.00,0.00
2025-07-21 04:01:00,223.67,6192.00,04:01:00,2025-07-21,13850.00,0.00
2025-07-21 04:02:00,224.06,5441.00,04:02:00,2025-07-21,19291.00,0.00
2025-07-21 04:03:00,224.24,4581.00,04:03:00,2025-07-21,23872.00,0.00
2025-07-21 04:04:00,224.30,8406.00,04:04:00,2025-07-21,32278.00,0.00
...,...,...,...,...,...,...
2025-08-18 19:55:00,143.02,396.00,19:55:00,2025-08-18,11779087.00,1.00
2025-08-18 19:56:00,143.02,499.00,19:56:00,2025-08-18,11779586.00,1.00
2025-08-18 19:57:00,143.01,219.00,19:57:00,2025-08-18,11779805.00,1.00
